In [57]:
## Importing required packages

import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle

In [58]:
## Importing the data

data = pd.read_csv('1-input/churn_base_data.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [59]:
## Pre-process the data
## Drop the irrelivant features

data = data.drop(['CustomerId', 'RowNumber', 'Surname'], axis=1)

In [60]:
## Encode the categorical variables - gender

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [61]:
## Encode the categorical variables - geography
## One hot encoder for muticlass category

onehot_geo = OneHotEncoder()
geo_encoded = onehot_geo.fit_transform(data[['Geography']])
onehot_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [62]:
## Creating a dataframe for geo encoded data

geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=onehot_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [63]:
## Combining the columns to the original data

df = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [64]:
## Dividing the dataset into independent and dependent features

X = df.drop('Exited', axis=1)
y = df['Exited']

## Split the data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scale these features
scalar = StandardScaler()
X_train_scaled = scalar.fit_transform(X_train)
X_test_scaled = scalar.transform(X_test)

In [65]:
## Save the encoders and scaler

with open('1-input/label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('1-input/onehot_encoder_geography.pkl', 'wb') as file:
    pickle.dump(onehot_geo, file)

In [66]:
## Save the encoders and scaler

with open('1-input/scalar.pkl', 'wb') as file:
    pickle.dump(scalar, file)

### ANN Implementation

In [67]:
## Importing required packages for modelling

import tensorflow as tf
from tensorflow import keras
import datetime

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [68]:
## Building the ANN model

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)), ## Hidden Layer 1 - Connected with input layer
    Dense(32, activation='relu'), ## Hidden Layer 2
    Dense(1, activation='sigmoid') ## Output Layer
])

/Users/mypc/Desktop/learning-gen-ai-udemy/04. Neural Network/.venv-ann/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [69]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [70]:
## Defining the optimizer and loss function
## Using Adam optimizer with a learning rate of 0.01 
## Binary cross entropy for loss function

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [71]:
## Compile the model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [72]:
## Set up the tensorboard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tf_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [73]:
## Set up early stopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [74]:
## Training the model 

history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[tf_callback, early_stopping]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step - accuracy: 0.8095 - loss: 0.4469 - val_accuracy: 0.8290 - val_loss: 0.3953
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - accuracy: 0.8439 - loss: 0.3828 - val_accuracy: 0.8600 - val_loss: 0.3555
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 469us/step - accuracy: 0.8572 - loss: 0.3515 - val_accuracy: 0.8600 - val_loss: 0.3414
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - accuracy: 0.8576 - loss: 0.3433 - val_accuracy: 0.8570 - val_loss: 0.3393
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step - accuracy: 0.8624 - loss: 0.3370 - val_accuracy: 0.8605 - val_loss: 0.3378
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - accuracy: 0.8631 - loss: 0.3335 - val_accuracy: 0.8615 - val_loss: 0.3362
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - accuracy: 0.8644 - loss: 0.3309 - val_accuracy: 0.8615 - val_loss: 0.3337
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - accuracy: 0.8649 - loss: 0

In [75]:
## Saving the model for tensor board

model.save('3-output/churn_model.h5')

In [76]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [77]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 16036), started 2 days, 20:41:32 ago. (Use '!kill 16036' to kill it.)